<a href="https://colab.research.google.com/github/HerGizHappa/Grok3Mini/blob/main/gui_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API-Einbindung mit Grok 3 Mini und NiceGUI**

In [ ]:
# Installiere die benötigten Bibliotheken
!pip install -U nicegui xai-sdk

# Installiere pyngrok für die Verbindung von NiceGUI in Colab
!pip install pyngrok

# Lade das deutsche Sprachmodell für spaCy (falls später für ABSA oder Syn-Chain benötigt)
!python -m spacy download de_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.6/156.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 69.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting

## Import relevanter Bibliotheken

In [ ]:
from nicegui import ui
from xai_sdk import Client
from xai_sdk.chat import user, system
import os


## Erstellung Client Objekts mit API Zugriff:

In [ ]:
client = Client(api_key=os.getenv("XAI_API_KEY")) #API-Key wurde als Umgebungsvariable gesetzt

Systemprompt hinzufügen, um zu bestimmen, wie das Sprachmodell die Anfrage des Users verarbeiten soll:

In [ ]:
chat = client.chat.create(model="grok-3-mini")  #Erstellung eines Chat-Objektes (global für Multi-Turn)
chat.append(system("Du bist Grok, ein hilfreicher Assistent von xAI.")) #system-Prompt eventuell später umändern für absa

In [ ]:
def askGrok():
    prompt = prompt_input.value.strip()
    if not prompt:
        output_label.value = 'Todo: Prompt'
        return #Verhinderung unnötiger API Anfrage bei leerem Prompt

    try:
        # User Prompt
        chat.append(user(prompt))

        # Generierung von Response (non-streaming)
        response = chat.sample()

        # Ausgabe: response.content ist der String
        output_text = response.content if hasattr(response, 'content') else str(response)

        output_label.value = output_text
        print("Ausgabe:", output_text)

        # Für Multi-Turn: Append Response zurück (z. B. in globalem Chat)
        # chat.append(response)

    except Exception as e:
        output_label.value = f'Fehler: {str(e)}'
        print(f'Fehler bei API-Aufruf: {str(e)}')

prompt_input = ui.textarea(label='API Prompt', placeholder='Frag Grok3 mini')
ui.button('Frage Grok', on_click=askGrok)
output_label = ui.textarea(placeholder='Grok-Antwort erscheint hier...')

ui.run(port=8081)